In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from PIL import Image
import os
import numpy as np

import mlflow
import mlflow.pytorch

In [15]:
class FaceDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Load the labels from the file
labels = np.loadtxt('dataset/label_train.txt', dtype=int)

# Prepare the image paths
image_dir = 'dataset/train_img/'
image_paths = [os.path.join(image_dir, f"{i+1:06d}.jpg") for i in range(len(labels))]

# Split the dataset into training and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(image_paths, labels, test_size=0.2, stratify=labels)

# Data augmentation and normalization for training
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = FaceDataset(train_paths, train_labels, transform=train_transform)
val_dataset = FaceDataset(val_paths, val_labels, transform=val_transform)


BATCH_SIZE = 1024
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=True, num_workers=4)


In [9]:
from torchvision.models import ResNet18_Weights

# Load a pre-trained ResNet18 model
model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

# Modify the fully connected layer to match our binary classification problem
model.fc = nn.Linear(model.fc.in_features, 1)

# Move the model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [16]:
LR = 0.001

optimizer = optim.Adam(model.parameters(), lr=LR)

# BCE with no weights

In [ ]:
# Start an MLflow experiment
mlflow.start_run()

# Log parameters
mlflow.log_param("learning_rate", LR)
mlflow.log_param("batch_size", BATCH_SIZE)

In [12]:
def eval_model(model, val_loader):
    model.eval()
    val_loss = 0.0
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device).float().view(-1, 1)
    
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    
            # Convert outputs to probabilities and then to binary predictions
            probs = torch.sigmoid(outputs)
            predicted = probs.round()
    
            # Store predictions and true labels for later analysis
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Convert lists to numpy arrays for sklearn functions
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    
    # Calculate and print the classification report
    print(classification_report(all_labels, all_preds, target_names=['Class 0', 'Class 1']))
    
    # Calculate False Acceptance Rate (FAR) and False Rejection Rate (FRR)
    false_acceptance = np.sum((all_labels == 0) & (all_preds == 1))
    false_rejection = np.sum((all_labels == 1) & (all_preds == 0))
    total_acceptance = np.sum(all_labels == 0)
    total_rejection = np.sum(all_labels == 1)
    
    far = false_acceptance / total_acceptance
    frr = false_rejection / total_rejection
    
    # Calculate the Half-Total Error Rate (HTER)
    hter = (far + frr) / 2
    print(f"Half-Total Error Rate (HTER): {hter:.4f}")
    mlflow.log_metric("HTER", hter, step=epoch)
    
    # Print average validation loss
    print(f"Validation Loss: {val_loss/len(val_loader):.4f}")
    mlflow.log_metric("val_loss", val_loss/len(val_loader), step=epoch)

In [17]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs):
    # Training loop
    num_epochs = 10
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
    
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device).float().view(-1, 1)  # Convert labels to float for BCE
    
            # Zero the parameter gradients
            optimizer.zero_grad()
    
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
    
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item()
    
        # Print training loss
        train_loss = running_loss/len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}")
        mlflow.log_metric("train_loss", train_loss, step=epoch)
    
        eval_model(model, val_loader)

In [4]:
# Calculate class weights based on the training data
class_counts = np.bincount(train_labels)
class_weights = 1. / class_counts
# class_weights = torch.tensor([len(train_labels) / sum(train_labels), len(train_labels) / (len(train_labels) - sum(train_labels))], dtype=torch.float32)

weights = torch.tensor([class_weights[0], class_weights[1]], dtype=torch.float32).to(device) *1e5


# Use BCEWithLogitsLoss which combines a Sigmoid layer and the BCELoss in one single class.
criterion = nn.BCEWithLogitsLoss(pos_weight=weights[1])




In [5]:
from sklearn.metrics import classification_report

# Old

In [7]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device).float().view(-1, 1)  # Convert labels to float for BCE

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print training loss
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

    model.eval()
    val_loss = 0.0
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device).float().view(-1, 1)
    
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    
            # Convert outputs to probabilities and then to binary predictions
            probs = torch.sigmoid(outputs)
            predicted = probs.round()
    
            # Store predictions and true labels for later analysis
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Convert lists to numpy arrays for sklearn functions
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    
    # Calculate and print the classification report
    print(classification_report(all_labels, all_preds, target_names=['Class 0', 'Class 1']))
    
    # Calculate False Acceptance Rate (FAR) and False Rejection Rate (FRR)
    false_acceptance = np.sum((all_labels == 0) & (all_preds == 1))
    false_rejection = np.sum((all_labels == 1) & (all_preds == 0))
    total_acceptance = np.sum(all_labels == 0)
    total_rejection = np.sum(all_labels == 1)
    
    far = false_acceptance / total_acceptance
    frr = false_rejection / total_rejection
    
    # Calculate the Half-Total Error Rate (HTER)
    hter = (far + frr) / 2
    print(f"Half-Total Error Rate (HTER): {hter:.4f}")
    
    # Print average validation loss
    print(f"Validation Loss: {val_loss/len(val_loader):.4f}")

Epoch [1/10], Loss: 0.1060
              precision    recall  f1-score   support

     Class 0       0.84      0.71      0.77      2420
     Class 1       0.96      0.98      0.97     17580

    accuracy                           0.95     20000
   macro avg       0.90      0.85      0.87     20000
weighted avg       0.95      0.95      0.95     20000

Half-Total Error Rate (HTER): 0.1520
Validation Loss: 0.1357
Epoch [2/10], Loss: 0.0991
              precision    recall  f1-score   support

     Class 0       0.85      0.71      0.77      2420
     Class 1       0.96      0.98      0.97     17580

    accuracy                           0.95     20000
   macro avg       0.90      0.85      0.87     20000
weighted avg       0.95      0.95      0.95     20000

Half-Total Error Rate (HTER): 0.1544
Validation Loss: 0.1404
Epoch [3/10], Loss: 0.0932
              precision    recall  f1-score   support

     Class 0       0.81      0.76      0.79      2420
     Class 1       0.97      0.98 

# One class